## Loading data

In [25]:
# Loading the IPython extention autoreload.
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Reloading edited module without restarting the kernel.
# Referring to https://support.enthought.com/hc/en-us/articles/204469240-Jupyter-IPython-After-editing-a-module-changes-are-not-effective-without-kernel-restart
%autoreload

from utils import *

path_to_train_and_dev = '../../data/first-round/train_and_dev_set'
path_to_test = '../../data/first-round/test_set'
ratio_of_training_data = 0.8
train_set_x, train_set_y, dev_set_x, dev_set_y, test_set_x = load_data(path_to_train_and_dev, ratio_of_training_data, path_to_test)

In [27]:
print('train x:', len(train_set_x), 'train y', len(train_set_y), 'dev x:', len(
    dev_set_x), 'dev y:', len(dev_set_y), 'test x:', len(test_set_x))

train x: 32 train y 32 dev x: 8 dev y: 8 test x: 10


## Loading and compiling model

In [49]:
%autoreload

from model import *

# Creating model.
model = create_model(input_shape=(train_set_x.shape[1:]), summary=True)

# Instantiating the optimizer.
opt = Adam()

# Determining the loss and metric, and compiling the model.
model.compile(optimizer=opt, loss='mean_squared_error')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 7500, 4, 1)        0         
_________________________________________________________________
batch_normalization_81 (Batc (None, 7500, 4, 1)        4         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 7497, 1, 64)       1088      
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 1874, 1, 64)       0         
_________________________________________________________________
batch_normalization_82 (Batc (None, 1874, 1, 64)       256       
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 1871, 1, 128)      32896     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 467, 1, 128)       0         
__________

In [33]:
#Loading weights, if needed.
#model.load_weights("./saved_weights/model-019epochs-0.028val_loss.h5")

## Fitting

In [50]:
# checkpoint
from keras.callbacks import ModelCheckpoint
filepath="./saved_weights/weights-{epoch:003d}epochs-{loss:.3f}loss-{val_loss:.3f}val_loss.h5"
checkpoint = ModelCheckpoint(filepath, save_weights_only=True)
callbacks_list = [checkpoint]

In [51]:
History_fit = model.fit(train_set_x, train_set_y, batch_size=train_set_x.shape[0], epochs=50, callbacks=callbacks_list, validation_data=(dev_set_x, dev_set_y))
history_fit = History_fit.history

Train on 32 samples, validate on 8 samples
Epoch 1/50
32/32 [==============================] - 8s 239ms/step - loss: 0.3322 - val_loss: 0.3253
Epoch 2/50
32/32 [==============================] - 2s 72ms/step - loss: 0.2536 - val_loss: 0.3253
Epoch 3/50
32/32 [==============================] - 2s 74ms/step - loss: 0.2301 - val_loss: 0.3253
Epoch 4/50
32/32 [==============================] - 3s 80ms/step - loss: 0.2119 - val_loss: 0.3253
Epoch 5/50
32/32 [==============================] - 2s 78ms/step - loss: 0.1978 - val_loss: 0.3253
Epoch 6/50
32/32 [==============================] - 2s 71ms/step - loss: 0.1944 - val_loss: 0.3253
Epoch 7/50
32/32 [==============================] - 3s 80ms/step - loss: 0.1924 - val_loss: 0.3253
Epoch 8/50
32/32 [==============================] - 2s 73ms/step - loss: 0.1892 - val_loss: 0.3253
Epoch 9/50
32/32 [==============================] - 2s 70ms/step - loss: 0.1801 - val_loss: 0.3253
Epoch 10/50
32/32 [==============================] - 3s 84ms/step

In [ ]:
# Saving history
#import pickle
#with open('./trainHistoryDict', 'wb') as file_pi:
#    pickle.dump(history_fit, file_pi)

## Evaluating the training and dev losses

In [ ]:
import pickle
with open('./trainHistoryDict', 'rb') as file_pi:
    history_fit = pickle.load(file_pi)

In [ ]:
# Calculating and printing root mean square error of fitting
import numpy as np
rms_train_w_dp_all = np.sqrt(history_fit['loss'])
rms_dev_all = np.sqrt(history_fit['val_loss'])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rms_train_w_dp_all)
plt.plot(rms_dev_all)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
# Determining the best epoch.
min_epoch = np.argmin(rms_dev_all)
print('Lowest loss of dev set occurs at epoch of', min_epoch)

# Loading the corresponding model.
model.load_weights("./saved_weights/weights-019epochs-0.093loss-0.029val_loss.h5")

# Evaluating training set without dropout layer.
ms_train_wo_dp_best = model.evaluate(x=train_set_x, y=train_set_y, batch_size=train_set_x.shape[0])
rms_train_wo_dp_best = np.sqrt(ms_train_wo_dp_best)

print('RMS error of training set of the best fitting:', rms_train_wo_dp_best)
print('RMS error of dev set of the best fitting:', rms_dev_all[min_epoch])

## Predicting

In [ ]:
prediction = model.predict(x=test_set_x)
print(prediction)